In [20]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [21]:
import pandas as pd
# train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
#                                   batch_size=-1, as_supervised=True)

# train_examples, train_labels = tfds.as_numpy(train_data)
# test_examples, test_labels = tfds.as_numpy(test_data)
data = pd.read_csv("./train.csv")
train_examples = np.array(data["text"])
train_labels = np.array(data["target"])
test_examples = np.array(data["text"])
test_labels = np.array(data["target"])

In [22]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 7613, test entries: 7613


In [24]:
model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

2022-05-14 23:16:39.335220: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


In [25]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
history = model.fit(train_examples,
                    train_labels,
                    epochs=100,
                    batch_size=32,
                    validation_data=(test_examples, test_labels),
                    verbose=1)

Epoch 1/100
82/82 [==============================] - 3s 33ms/step - loss: 0.6054 - accuracy: 0.7210 - val_loss: 0.5868 - val_accuracy: 0.7154
Epoch 2/100
82/82 [==============================] - 3s 32ms/step - loss: 0.4318 - accuracy: 0.8320 - val_loss: 0.5412 - val_accuracy: 0.7328
Epoch 3/100
82/82 [==============================] - 3s 32ms/step - loss: 0.2906 - accuracy: 0.8925 - val_loss: 0.5368 - val_accuracy: 0.7354
Epoch 4/100
82/82 [==============================] - 3s 32ms/step - loss: 0.1737 - accuracy: 0.9418 - val_loss: 0.5682 - val_accuracy: 0.7214
Epoch 5/100
82/82 [==============================] - 3s 33ms/step - loss: 0.0966 - accuracy: 0.9736 - val_loss: 0.6247 - val_accuracy: 0.7180
Epoch 6/100
82/82 [==============================] - 3s 32ms/step - loss: 0.0541 - accuracy: 0.9866 - val_loss: 0.6831 - val_accuracy: 0.7120
Epoch 7/100
82/82 [==============================] - 3s 33ms/step - loss: 0.0348 - accuracy: 0.9897 - val_loss: 0.7352 - val_accuracy: 0.7182
Epoch 

In [29]:
results = model.evaluate(test_examples, test_labels)

print(results)

238/238 [==============================] - 1s 4ms/step - loss: 1.1021 - accuracy: 0.7858
[1.1021066904067993, 0.7857611775398254]
